In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from copy import deepcopy
import nltk
import pickle
flatten = lambda l: [item for sublist in l for item in sublist]
from data_utils import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
data = open("train.iob","r",encoding="utf-8").readlines()

In [6]:
train_data=[]
history=[["<null>"]]
for d in data:
    if d=="\n":
        history=[["<null>"]]
        continue
    dd = d.replace("\n","").split("|||")
    if len(dd)==1:
        bot = tagger.morphs(dd[0])
        history.append(bot)
    else:
        user = dd[0].split()
        tag = dd[1].split()
        intent = dd[2]
        temp = deepcopy(history)
        train_data.append([temp,user,tag,intent])
        history.append(user)

In [12]:
train_data[2]

[[['<null>'],
  ['will', 'it', 'rain', 'this', 'week'],
  ['What',
   'city',
   'are',
   'you',
   'wanting',
   'to',
   'know',
   'if',
   'it',
   "'",
   's',
   'going',
   'to',
   'rain',
   '?']],
 ['Menlo', 'Park'],
 ['B-location', 'I-location'],
 'weather']

In [13]:
historys, currents, slots, intents = list(zip(*train_data))

In [14]:
vocab = list(set(flatten(currents)))
slot_vocab = list(set(flatten(slots)))
intent_vocab = list(set(intents))

In [15]:
word2index={"<pad>" : 0, "<unk>" : 1, "<null>" : 2, "<s>" : 3, "</s>" : 4}
for vo in vocab:
    if word2index.get(vo)==None:
        word2index[vo] = len(word2index)
        
slot2index={"<pad>" : 0}
for vo in slot_vocab:
    if slot2index.get(vo)==None:
        slot2index[vo] = len(slot2index)
        
intent2index={}
for vo in intent_vocab:
    if intent2index.get(vo)==None:
        intent2index[vo] = len(intent2index)

In [21]:
for t in train_data:
    for i,history in enumerate(t[0]):
        t[0][i] = prepare_sequence(history, word2index).view(1, -1)

    t[1] = prepare_sequence(t[1], word2index).view(1, -1)
    t[2] = prepare_sequence(t[2], slot2index).view(1, -1)
    t[3] = torch.LongTensor([intent2index[t[3]]]).view(1,-1)

In [22]:
class SDEN(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,slot_size,intent_size):
        super(SDEN,self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embed_size)
        self.bigru_m = nn.GRU(embed_size,hidden_size,batch_first=True,bidirectional=True)
        self.bigru_c = nn.GRU(embed_size,hidden_size,batch_first=True,bidirectional=True)
        self.context_encoder = nn.Sequential(nn.Linear(hidden_size*4,hidden_size*2),
                                                               nn.Sigmoid())
        self.session_encoder = nn.GRU(hidden_size*2,hidden_size*2,batch_first=True,bidirectional=True)
        
        self.decoder_1 = nn.GRU(embed_size,hidden_size*2,batch_first=True,bidirectional=True)
        self.decoder_2 = nn.LSTM(hidden_size*4,hidden_size*2,batch_first=True,bidirectional=True)
        
        self.intent_linear = nn.Linear(hidden_size*4,intent_size)
        self.slot_linear = nn.Linear(hidden_size*4,slot_size)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self,history,current):
        batch_size = len(history)
        H= [] # encoded history
        for h in history:
            mask = h.eq(0)
            embeds = self.embed(h)
            embeds = self.dropout(embeds)
            outputs, hidden = self.bigru_m(embeds)
            real_hidden = []

            for i, o in enumerate(outputs): # B,T,D
                real_length = mask[i].data.tolist().count(0) 
                real_hidden.append(o[real_length - 1])

            H.append(torch.cat(real_hidden).view(h.size(0), -1).unsqueeze(0))
        
        M = torch.cat(H) # B,T_C,2H
        M = self.dropout(M)
        embeds = self.embed(current)
        embeds = self.dropout(embeds)
        mask = current.eq(0)
        outputs, hidden = self.bigru_c(embeds)
        real_hidden=[]
        for i, o in enumerate(outputs): # B,T,D
            real_length = mask[i].data.tolist().count(0) 
            real_hidden.append(o[real_length - 1])
        C = torch.cat(real_hidden).view(current.size(0),1, -1) # B,1,2H
        C = self.dropout(C)
        
        CONCAT = []
        for i in range(batch_size):
            m = M[i] # T_c,2H
            c = C[i] # 1,2H
            c = c.expand_as(m)
            cat = torch.cat([m,c],1)
            CONCAT.append(cat.unsqueeze(0))
        CONCAT = torch.cat(CONCAT)
        
        G = self.context_encoder(CONCAT)
        
        _,H = self.session_encoder(G) # 2,B,2H
        weight = next(self.parameters())
        cell_state = weight.new_zeros(H.size())
        O_1,_ = self.decoder_1(embeds)
        O_1 = self.dropout(O_1)
        
        O_2,(S_2,_) = self.decoder_2(O_1,(H,cell_state))
        O_2 = self.dropout(O_2)
        S = torch.cat([s for s in S_2],1)
        
        intent_prob = self.intent_linear(S)
        slot_prob = self.slot_linear(O_2.contiguous().view(O_2.size(0)*O_2.size(1),-1))
        
        return slot_prob, intent_prob

In [23]:
EPOCH = 20
BATCH = 32
LR = 0.001

In [24]:
EPOCH = 10

In [25]:
model = SDEN(len(word2index),100,64,len(slot2index),len(intent2index))
slot_loss_function = nn.CrossEntropyLoss(ignore_index=0)
intent_loss_function = nn.CrossEntropyLoss()
model.to(device)
optimizer = optim.Adam(model.parameters(),lr=LR)
scheduler = optim.lr_scheduler.MultiStepLR(gamma=0.1,milestones=[EPOCH//4,EPOCH//2],optimizer=optimizer)

In [26]:
model.train()
for epoch in range(EPOCH):
    losses=[]
    scheduler.step()
    for i,batch in enumerate(data_loader(train_data,BATCH,True)):
        h,c,slot,intent = pad_to_batch(batch,word2index,slot2index)
        h = [hh.to(device) for hh in h]
        c = c.to(device)
        slot = slot.to(device)
        intent = intent.to(device)
        model.zero_grad()
        slot_p, intent_p = model(h,c)

        loss_s = slot_loss_function(slot_p,slot.view(-1))
        loss_i = intent_loss_function(intent_p,intent.view(-1))
        loss = loss_s + loss_i
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
        if i % 20 == 0:
            print("[%d/%d] [%d/%d] mean_loss : %.3f" % (epoch,EPOCH,i,len(train_data)//BATCH,np.mean(losses)))
            losses=[]

[0/10] [0/961] mean_loss : 4.612
[0/10] [20/961] mean_loss : 3.109
[0/10] [40/961] mean_loss : 2.019
[0/10] [60/961] mean_loss : 1.889
[0/10] [80/961] mean_loss : 1.654
[0/10] [100/961] mean_loss : 1.527
[0/10] [120/961] mean_loss : 1.458
[0/10] [140/961] mean_loss : 1.335
[0/10] [160/961] mean_loss : 1.174
[0/10] [180/961] mean_loss : 1.053
[0/10] [200/961] mean_loss : 1.052
[0/10] [220/961] mean_loss : 1.008
[0/10] [240/961] mean_loss : 1.024
[0/10] [260/961] mean_loss : 0.959
[0/10] [280/961] mean_loss : 0.937
[0/10] [300/961] mean_loss : 0.932
[0/10] [320/961] mean_loss : 0.826
[0/10] [340/961] mean_loss : 0.827
[0/10] [360/961] mean_loss : 0.874
[0/10] [380/961] mean_loss : 0.836
[0/10] [400/961] mean_loss : 0.867
[0/10] [420/961] mean_loss : 0.773
[0/10] [440/961] mean_loss : 0.742
[0/10] [460/961] mean_loss : 0.779
[0/10] [480/961] mean_loss : 0.705
[0/10] [500/961] mean_loss : 0.692


KeyboardInterrupt: 

In [27]:
import json

In [28]:
data = json.load(open('../dataset/kvret/kvret_dev_public.json','r'))

In [29]:
train_data[0]

[[tensor([[ 2]]),
  tensor([[ 1131,   640,  1126,   640,   555,   284,   953,   698,   400,
             990,   640,   555,  1149,   283,    68,   530,  1077]]),
  tensor([[    1,   990,   670,   159,  1109,   673,   944,   236,   698,
             382,   593,   595,   436,   542,   514]]),
  tensor([[ 1042,   555,  1126,   698,  1109,   990,  1077]]),
  tensor([[ 1118,   539,     1,   509,   782,   326,    70,   514]])],
 tensor([[ 983,  640,  222,  251,  471]]),
 tensor([[ 16,  16,  16,  16,  16]]),
 tensor([[ 2]])]

In [30]:
index2slot = {v:k for k,v in slot2index.items()}
index2intent = {v:k for k,v in intent2index.items()}

In [38]:
index = random.choice(range(len(data)))

In [49]:
index = random.choice(range(len(data)))
history=[prepare_sequence(["<null>"],word2index).view(1,-1)]
for d in data[index]['dialogue']:
    
    if d['turn']=='assistant':
        phrase = nltk.word_tokenize(d['data']['utterance'])
        phrase = prepare_sequence(phrase,word2index).view(1,-1)
        history.append(phrase)
    else:
        h = pad_to_history(history,word2index)
        c = nltk.word_tokenize(d['data']['utterance'])
        c = prepare_sequence(c,word2index).view(1,-1)
        with torch.no_grad():
            slot_p, intent_p = model(h,c)
        
        slots = slot_p.max(1)[1]
        intent = intent_p.max(1)[1]
        slots = [index2slot[i] for i in slots.tolist()]
        intent = index2intent[intent.item()]
        print(d['data']['utterance'])
        print(slots)
        print(intent)
        print("\n")
        history.append(c)

Schedule my 10 am dentist appointment on Friday with my father.
['O', 'O', 'B-event', 'O', 'I-event', 'I-event', 'O', 'B-date', 'O', 'O', 'O', 'O']
schedule


List my current schedule. 
['O', 'O', 'O', 'O', 'O']
navigate


List my schedule for Friday.
['O', 'O', 'O', 'O', 'O', 'O']
schedule


Thank you so much!
['O', 'O', 'O', 'O', 'O']
thanks




In [35]:
import pickle

In [36]:
model = model.cpu()
torch.save(model.state_dict(),'sden.pkl')

In [37]:
pickle.dump(word2index,open('vocab.pkl','wb'))
pickle.dump(slot2index,open('slot.pkl','wb'))
pickle.dump(intent2index,open('intent.pkl','wb'))